In [ ]:
# Copyright 2026 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden - Recursion MAE Image Feature Extraction Deployment

<table><tbody><tr>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fcommunity%2Fmodel_garden%2Fmodel_garden_recursion_mae_pytorch_inference_deployment.ipynb">
      <img alt="Google Cloud Colab Enterprise logo" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" width="32px"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_recursion_mae_pytorch_inference_deployment.ipynb">
      <img alt="GitHub logo" src="https://github.githubassets.com/assets/GitHub-Mark-ea2971cee799.png" width="32px"><br> View on GitHub
    </a>
  </td>
</tr></tbody></table>

## Overview

This notebook demonstrates deploying [recursionpharma/OpenPhenom](https://huggingface.co/recursionpharma/OpenPhenom) with the VMG Pytorch Inference container. In additional to `recursionpharma/OpenPhenom `, You can view and change the code to deploy a different Hugging Face Pytorch model with appropriate machine specs. 


### OpenPhenom Model Licensing

* OpenPhenom Model is available under a Non-Commercial End User License Agreement license. For full details, please refer to the [license documentation](https://huggingface.co/recursionpharma/OpenPhenom/blob/main/LICENSE) that governs the use of this model.


### Objective

- Download and deploy the `recursionpharma/OpenPhenom` model with VMG Pytorch Inference container
- Send prediction request to the deployed endpoint

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Run the notebook

In [ ]:
# @title Setup Google Cloud project

# @markdown 1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

# @markdown 2. **[Optional]** Set region. If not set, the region will be set automatically according to Colab Enterprise environment.

REGION = ""  # @param {type:"string"}

# Upgrade Vertex AI SDK.
! pip3 install --upgrade --quiet 'google-cloud-aiplatform>=1.64.0'
! git clone https://github.com/GoogleCloudPlatform/vertex-ai-samples.git

import importlib
import os

from google.cloud import aiplatform

common_util = importlib.import_module(
    "vertex-ai-samples.notebooks.community.model_garden.docker_source_codes.notebook_util.common_util"
)

# Get the default cloud project id.
PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]

# Get the default region for launching jobs.
if not REGION:
    if not os.environ.get("GOOGLE_CLOUD_REGION"):
        raise ValueError(
            "REGION must be set. See"
            " https://cloud.google.com/vertex-ai/docs/general/locations for"
            " available cloud locations."
        )
    REGION = os.environ["GOOGLE_CLOUD_REGION"]

# Enable the Vertex AI API and Compute Engine API, if not already.
print("Enabling Vertex AI API and Compute Engine API.")
! gcloud services enable aiplatform.googleapis.com compute.googleapis.com

# Initialize Vertex AI API.
print("Initializing Vertex AI API.")
aiplatform.init(project=PROJECT_ID, location=REGION)
! gcloud config set project $PROJECT_ID

models, endpoints = {}, {}

In [ ]:
# @title Deploy the model to Vertex for online predictions

# @markdown This section uploads the model [recursionpharma/OpenPhenom](https://huggingface.co/recursionpharma/OpenPhenom) to Model Registry and deploys it on the Endpoint using a CPU machine.

# @markdown The deployment takes ~15 minutes to finish.

MODEL_ID = "recursionpharma/OpenPhenom"  # @param {type: "string", isTemplate: true}
TASK = "image-feature-extraction-mae"  # @param {type: "string", isTemplate: true}
MACHINE_TYPE = "e2-standard-4"
# The pre-built serving docker image. It contains serving scripts and models.
SERVE_DOCKER_URI = "us-docker.pkg.dev/deeplearning-platform-release/vertex-model-garden/pytorch-inference.cu125.0-2.ubuntu2204.py310"


def deploy_model(model_id: str, task: str):
    """Create a Vertex AI Endpoint and deploy the specified model to the endpoint."""
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_id}-endpoint")
    serving_env = {
        "MODEL_ID": model_id,
        "TASK": task,
        "DEPLOY_SOURCE": "notebook",
    }

    model = aiplatform.Model.upload(
        display_name=model_id,
        serving_container_image_uri=SERVE_DOCKER_URI,
        serving_container_ports=[7080],
        serving_container_predict_route="/predict",
        serving_container_health_route="/health",
        serving_container_environment_variables=serving_env,
    )

    model.deploy(
        endpoint=endpoint,
        machine_type=MACHINE_TYPE,
        deploy_request_timeout=1800,
        service_account="",
        system_labels={
            "NOTEBOOK_NAME": "model_garden_recursion_mae_pytorch_inference_deployment.ipynb"
        },
    )
    return model, endpoint


models["pytorch_serve"], endpoints["pytorch_serve"] = deploy_model(
    model_id=MODEL_ID,
    task=TASK,
)
print("endpoint_name:", endpoints["pytorch_serve"].name)

# @markdown Click "Show Code" to see more details.

In [ ]:
# @title Predict
import pandas as pd

SAMPLE_IMAGES_GCS_PATH = "gs://cloud-samples-data/vertex-ai/model-garden/recursion-mae-images"  # @param {type: "string", isTemplate: true}

instances = [{"gcs_uri": SAMPLE_IMAGES_GCS_PATH}]
response = endpoints["pytorch_serve"].predict(instances=instances)

final_df = pd.DataFrame(response.predictions)
display(final_df)

## Clean up resources

In [ ]:
# @title Delete the models and endpoints
# @markdown  Delete the experiment models and endpoints to recycle the resources
# @markdown  and avoid unnecessary continuous charges that may incur.

# Undeploy model and delete endpoint.
for endpoint in endpoints.values():
    endpoint.delete(force=True)

# Delete models.
for model in models.values():
    model.delete()